# Ledger - World Cup Betting Demo - SQL Server 2022

## Append-Only Ledger Table Bets

In [ ]:
CREATE TABLE [dbo].[Bets](
	[BetID] [INT] IDENTITY(1,1) NOT NULL,
	[MoneylineID] [INT] NOT NULL,
	[FirstName] [NVARCHAR](50) NOT NULL,
	[LastName] [NVARCHAR](50) NOT NULL,
	[Country] [NVARCHAR](50),
	[Bet] [MONEY] NOT NULL,
	[Payout] [MONEY] NOT NULL,
	[BetDateTime] [DATETIME2] NOT NULL
	)
WITH (LEDGER = ON (APPEND_ONLY = ON));
GO

## Let's try to modify the bet

<span style="font-family: Calibri, sans-serif; font-size: 11pt;">The malicious DBA tries to manipulate the faulty record but noticed that it’s an append-only ledger table and that data cannot be modified.&nbsp;</span>

In [ ]:
USE WorldCup
GO
SELECT * from Bets
WHERE FirstName='Pieter' and Lastname='Vanhove'

In [ ]:
UPDATE Bets
SET Payout=-2400
WHERE BetID=5

## Different parties can verify the database to be sure the data can be trusted.

In [ ]:
DECLARE @digest_locations NVARCHAR(MAX) = (SELECT * FROM sys.database_ledger_digest_locations FOR JSON AUTO, INCLUDE_NULL_VALUES);
    SELECT @digest_locations as digest_locations;
    BEGIN TRY
        EXEC sys.sp_verify_database_ledger_from_digest_storage @digest_locations;
    SELECT 'Ledger verification succeeded.' AS Result;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH

## Let's tamper the data

DBA thinks he/she’s smart and tampers with the data directly into the data file by using a stored procedure.

In [ ]:
SELECT sys.fn_PhysLocFormatter(%%physloc%%) PageId, *
FROM Bets
WHERE BetID=5 --Copy the ID from the previous result set

In [ ]:
EXECUTE sp_TamperWithBet
	@PageID=568, 
	@ID=5,
	@PayOut=-2400

Show that the bet was changed

In [ ]:
SELECT * from Bets
WHERE FirstName='Pieter' and Lastname='Vanhove'

## Verify the database again

In [ ]:
DECLARE @digest_locations NVARCHAR(MAX) = (SELECT * FROM sys.database_ledger_digest_locations FOR JSON AUTO, INCLUDE_NULL_VALUES);
    SELECT @digest_locations as digest_locations;
    BEGIN TRY
        EXEC sys.sp_verify_database_ledger_from_digest_storage @digest_locations;
    SELECT 'Ledger verification succeeded.' AS Result;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH

## Review the history of the transaction

In [ ]:
SELECT
  t.[commit_time] AS [CommitTime] 
	, t.[principal_name] AS [UserName]
    ,l.[MoneylineID]
    ,l.[FirstName]
    ,l.[LastName]
    ,l.[Country]
    ,l.[Bet]
    ,l.[Payout]
    ,l.[BetDateTime]
	, l.[ledger_operation_type_desc] AS Operation
	FROM [dbo].[Bets_Ledger] l
	JOIN [sys].[database_ledger_transactions] t
	ON t.[transaction_id] = l.[ledger_transaction_id]
	WHERE t.transaction_id=1166;